In [ ]:
from numba import njit, jit
import numba
import numpy as np
import imutils
import cv2
from matplotlib import pyplot as plt

# from decimal import *
# getcontext().prec = 100000000

In [ ]:
path = "1.jpg"
alpha = 1

In [ ]:
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = imutils.resize(img, width=100)
height, width, _ = img.shape
print(height, width)
plt.figure(figsize=(20, 20))
plt.imshow(img)
plt.show()

In [ ]:
img = img.astype(np.float64)
img = img / 100.
x_up = img[0:int(height/4)]
x_down = img[int(3*height/4):height]

plt.figure(figsize=(20, 20))
plt.subplot(121), plt.imshow(x_up)
plt.subplot(122), plt.imshow(x_down)
plt.show()

In [ ]:
mu_up = np.sum(x_up, axis=0)
mu_up = np.sum(mu_up, axis=0)
mu_up = mu_up.astype(float) / (x_up.shape[0] * x_up.shape[1])
mu_up = np.array(mu_up, np.float64)
print(mu_up)
mu_down = np.sum(x_down, axis=0)
mu_down = np.sum(mu_down, axis=0)
mu_down = mu_down.astype(float) / (x_down.shape[0] * x_down.shape[1])
mu_down = np.array(mu_down, np.float64)
print(mu_down)

In [ ]:
matrix = [np.matmul([[(x - mu_up)[0]], [(x - mu_up)[1]], [(x - mu_up)[2]]], [(x - mu_up)]) for x_row in x_up for x in x_row]
epsilon_up = np.sum(matrix, axis=0)
epsilon_up = epsilon_up.astype(float) / (x_up.shape[0] * x_up.shape[1])
epsilon_up = np.array(epsilon_up, np.float64)
print(epsilon_up)
matrix = [np.matmul([[(x - mu_down)[0]], [(x - mu_down)[1]], [(x - mu_down)[2]]], [(x - mu_down)]) for x_row in x_down for x in x_row]
epsilon_down = np.sum(matrix, axis=0)
epsilon_down = epsilon_down.astype(float) / (x_down.shape[0] * x_down.shape[1])
epsilon_down = np.array(epsilon_down, np.float64)
print(epsilon_down)

In [ ]:
modKs = 3
modNei = 8
modT = height * width
mus = np.array([mu_down, (mu_up + mu_down) * 0.5, mu_up])
epsilons = np.array([epsilon_down, (epsilon_up + epsilon_down) * 0.5, epsilon_up])

## Дифузія

In [ ]:
@njit
def prod_vec(x, y):
    z0 = x[0] * y[0]
    z1 = x[1] * y[1]
    z2 = x[2] * y[2]
    return z0 + z1 + z2
@njit
def prod_matr(m, x):
    return [prod_vec(m[0], x), prod_vec(m[1], x), prod_vec(m[2], x)]
@njit
def q(x, mu, epsilon):
    y = x - mu
    z = prod_matr(epsilon, y)
    return -prod_vec(y, z)
@njit
def g(i, j, i_, j_, k, k_):
    if k == k_:
        if k != 1:
            return 0
        elif (j != j_):
            return 0
        else:
            return -np.inf
    
    if k - k_ > 1:
        if (i_ - i == 1) and (j != j_):
            return -alpha
        else:
            return -np.inf
        
    if k_ - k > 1:
        if (i - i_ == 1) and (j != j_):
            return -alpha
        else:
            return -np.inf
        
    if k - k_ == 1:
        if k == 1:
            if (i <= i_):
                return -alpha
            else:
                return -np.inf
        if k == 2:
            if (i <= i_):
                return -alpha
            else:
                return -np.inf
            
    if k_ - k == 1:
        if k_ == 1:
            if (i_ <= i):
                return -alpha
            else:
                return -np.inf
        if k_ == 2:
            if (i_ <= i):
                return -alpha
            else:
                return -np.inf
    return -np.inf
@njit
def get_neighbors(i, j, h, w):
    Neighbors = np.zeros((modNei, 2), numba.int32)
    Index = 0
    Nt = 0

    if (i != 0):
        if (j != 0):
            Neighbors[Index] = [i - 1, j - 1]
            Nt += 1
        else:
            Neighbors[Index] = [-1, -1]
        Index += 1

        Neighbors[Index] = [i - 1, j]
        Nt += 1
    else:
        Neighbors[Index] = [-1, -1]
    Index += 1

    if (j != 0):
        if (h - i != 1):
            Neighbors[Index] = [i + 1, j - 1]
            Nt += 1
        else:
            Neighbors[Index] = [-1, -1]
        Index += 1

        Neighbors[Index] = [i, j - 1]
        Nt += 1
    else:
        Neighbors[Index] = [-1, -1]
    Index += 1

    if (h - i != 1):
        if (w - j != 1):
            Neighbors[Index] = [i + 1, j + 1]
            Nt += 1
        else:
            Neighbors[Index] = [-1, -1]
        Index += 1

        Neighbors[Index] = [i + 1, j]
        Nt += 1
    else:
        Neighbors[Index] = [-1, -1]
    Index += 1

    if (w - j != 1):
        if (i != 0):
            Neighbors[Index] = [i - 1, j + 1]
            Nt += 1
        else:
            Neighbors[Index] = [-1, -1]
        Index += 1

        Neighbors[Index] = [i, j + 1]
        Nt += 1
    else:
        Neighbors[Index] = [-1, -1]

    return Neighbors, Nt

#############################################

@njit
def run(loops):
    phi = np.zeros((height, width, modNei, modKs))

    for interation in range(loops):
        if interation % 10 == 0:
            print(interation)
        for i in range(height):
            for j in range(width):
                for k in range(modKs):
                    nb, Nt = get_neighbors(i, j, height, width)
                    kStar = np.zeros(modNei, numba.int64)
                    for ind in range(modNei):
                        if (nb[ind][0] != -1):
                            masK = np.zeros(modKs, numba.float64)
                            for k_ in range(modKs):
                                masK[k_] = g(i, j, nb[ind][0], nb[ind][1], k, k_)
                                masK[k_] -= phi[i][j][ind][k]
                                masK[k_] -= phi[nb[ind][0]][nb[ind][1]][(ind + 4) % modNei][k_]
                            kStar[ind] = np.argmax(masK)

                    Ct = 0.
                    for ind in range(modNei):
                        if (nb[ind][0] != -1):
                            Ct += g(i, j, nb[ind][0], nb[ind][1], k, kStar[ind])
                            Ct -= phi[nb[ind][0]][nb[ind][1]][(ind + 4) % modNei][kStar[ind]]
                    Ct += q(img[i][j], mus[k], epsilons[k])
                    Ct /= Nt
                    for ind in range(modNei):
                        if (nb[ind][0] != -1):
                            phi[i][j][ind][k] = g(i, j, nb[ind][0], nb[ind][1], k, kStar[ind])
                            phi[i][j][ind][k] -= phi[nb[ind][0]][nb[ind][1]][(ind + 4) % modNei][kStar[ind]]
                            phi[i][j][ind][k] -= Ct


    raw = np.zeros((height, width), numba.int64)
    for i in range(height):
        for j in range(width):
            nb, Nt = get_neighbors(i, j, height, width)
            ind = 0
            while (nb[ind][0] == -1):
                ind += 1
            kMax = np.zeros(modKs, numba.float64)
            for k in range(modKs):
                masK = np.zeros(modKs, numba.float64)
                for k_ in range(modKs):
                    masK[k_] = g(i, j, nb[ind][0], nb[ind][1], k, k_) - phi[i][j][ind][k] - phi[nb[ind][0]][nb[ind][1]][(ind + 4) % modNei][k_]
                kMax[k] = np.max(masK)
            raw[i][j] = np.argmax(kMax)

    result = np.zeros((height, width, 3), numba.uint8)
    for i in range(height):
        for j in range(width):
            result[i][j] = mus[raw[i][j]] * 100

    return result

In [ ]:
result = run(40)
plt.figure(figsize=(20, 20))
plt.imshow(result)
plt.show()

cv2.imwrite("res_1.jpg", result) 

## Викреслювання і самоконтроль

In [ ]:
def cross(q_, g_):
    q = q_.copy()
    g = g_.copy()

    changed = True

    while changed:
        changed = False

        for i in range(81):
            for k in range(9):
                if q[i][k] == 1:
                    for i1 in neig[i]:
                        cur = 0
                        for k1 in range(9):
                            cur = cur or g[i*9+k][i1*9+k1]
                        if cur == 0:
                            q[i][k] = 0
                            changed = True
                            break   

        for i in range(81):
            for k in range(9):
                for i1 in neig[i]:
                    for k1 in range(9):
                        if g[i*9+k][i1*9+k1] == 1:
                            cur = q[i][k] and q[i1][k1]
                            g[i*9+k][i1*9+k1] = cur
                            if cur == 0:
                                changed = True

    return q, g

def f(q):
    res = 0
    for k in range(9):
        res += q[0][k]
    return int(res > 0)

def gamma(k):
    for i in range(81):
        if q[i][k[i]] == 0:
            return 0
        for i1 in neig[i]:
            if g[i*9+k[i]][i1*9+k1[i1]] == 0:
                return 0
    return 1

q_cur, g_cur = cross(q, g)
k_star = []

else:
    for i in range(81):
        found = False
        for k in range(9):
            if q_cur[i][k] == 1:
                q = q_cur.copy()
                g = g_cur.copy()
                for k_ in range(9):
                    if k != k_:
                        q[i][k_] = 0
                q_cur, g_cur = cross(q, g)
                if f(q_cur) == 1:
                    k_star.append(k)
                    found = True
                    break
if len(k_star) == 81:
    result = np.zeros((9, 9), np.int32)
    for i in range(9):
        for j in range(9):
            result[i][j] = k_star[kv[i*9+j]]+1
    print(result)